## Imports

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.12.0'

## Preprocessing



### Load dataset

In [5]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data  # we only take the first two features.
y = iris.target

print('X.shape:', X.shape)
print('y.shape:', y.shape)

print(X[0:5,:])
print(np.unique(y))

X.shape: (150, 4)
y.shape: (150,)
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 1 2]


In [9]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Cross validation

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

print('X_train.shape', X_train.shape)
print('y_train.shape', y_train.shape)
print('X_test.shape', X_test.shape)
print('y_test.shape', y_test.shape)

X_train.shape (90, 4)
y_train.shape (90,)
X_test.shape (60, 4)
y_test.shape (60,)


In [11]:
from keras.utils import to_categorical

y_train_oh = to_categorical(y_train)

print(y_train)
print(y_train_oh)

[0 2 1 0 0 2 1 2 2 0 1 1 2 0 2 1 1 0 2 2 0 0 1 1 2 0 0 1 0 1 2 0 2 0 0 1 0
 0 1 2 1 1 1 0 0 1 2 0 0 1 1 1 2 1 1 1 2 0 0 1 2 2 2 2 0 1 0 1 1 0 1 2 1 2
 2 0 1 0 2 2 1 1 2 2 1 0 1 1 2 2]
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]


### Data normalization



In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print('X_train mean', np.mean(X_train))
print('X_train standard deviation', np.std(X_train))

X_train mean 0.0
X_train standard deviation 0.9999999999999998


## Learning

### Creating the Adaline

In [14]:
from keras.activations import linear
X_input = tf.keras.layers.Input([4])
y_pred = tf.keras.layers.Dense(units=3, activation='sigmoid', kernel_initializer='zeros', use_bias='yes')(X_input)

model = tf.keras.models.Model(inputs=X_input, outputs=y_pred)

model.compile(loss='categorical_crossentropy', metrics='accuracy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.05))

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 3)                 15        
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


### Train

In [15]:
model.fit(X_train, y_train_oh, epochs=10, batch_size=5, verbose=1)

Epoch 1/10
18/18 [==============================] - 4s 10ms/step - loss: 0.8623 - accuracy: 0.7556
Epoch 2/10
18/18 [==============================] - 1s 3ms/step - loss: 0.6329 - accuracy: 0.8000
Epoch 3/10
18/18 [==============================] - 0s 2ms/step - loss: 0.5455 - accuracy: 0.8222
Epoch 4/10
18/18 [==============================] - 0s 2ms/step - loss: 0.4974 - accuracy: 0.8333
Epoch 5/10
18/18 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.8556
Epoch 6/10
18/18 [==============================] - 0s 2ms/step - loss: 0.4378 - accuracy: 0.8667
Epoch 7/10
18/18 [==============================] - 0s 2ms/step - loss: 0.4198 - accuracy: 0.8556
Epoch 8/10
18/18 [==============================] - 0s 2ms/step - loss: 0.4015 - accuracy: 0.8667
Epoch 9/10
18/18 [==============================] - 0s 2ms/step - loss: 0.3867 - accuracy: 0.8778
Epoch 10/10
18/18 [==============================] - 0s 2ms/step - loss: 0.3735 - accuracy: 0.8889


## Evaluation

In [17]:
y_test_oh = to_categorical(y_test)

In [18]:
test_loss, test_accuracy = model.evaluate(X_test, y_test_oh)

2/2 [==============================] - 2s 2s/step - loss: 0.2812 - accuracy: 0.9333


In [25]:
y_test_pred = model.predict(X_test)

print(y_test_pred)

2/2 [==============================] - 0s 147ms/step
[[0.10054874 0.78679276 0.7079486 ]
 [0.05993924 0.68166494 0.87986773]
 [0.05120759 0.7209757  0.87761056]
 [0.4014425  0.6511306  0.4440962 ]
 [0.9712001  0.44913998 0.03509362]
 [0.17557804 0.6599645  0.7075392 ]
 [0.1167815  0.82090217 0.6226465 ]
 [0.96192837 0.54769206 0.03165107]
 [0.9400891  0.63282794 0.03565765]
 [0.28142673 0.75123227 0.45814833]
 [0.06275441 0.64805967 0.89024   ]
 [0.9792834  0.30024436 0.0469875 ]
 [0.36209428 0.71248114 0.4155232 ]
 [0.0977013  0.68130356 0.812031  ]
 [0.02039588 0.7052954  0.95253664]
 [0.05747004 0.6773639  0.88651425]
 [0.9479961  0.48887214 0.05424301]
 [0.96140766 0.6007199  0.02598745]
 [0.21770525 0.6942339  0.6128009 ]
 [0.92508596 0.6129197  0.04865392]
 [0.9487525  0.51185536 0.04898992]
 [0.13498673 0.7256307  0.7078577 ]
 [0.9505054  0.5100938  0.04762914]
 [0.15938008 0.6636966  0.7277102 ]
 [0.96963507 0.39862096 0.04511334]
 [0.93329185 0.52183837 0.06146816]
 [0.9253826

In [26]:
y_test_pred.shape

(60, 3)

In [27]:
y_test_pred = np.argmax(y_test_pred, axis=1)

print(y_test_pred)

[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 1 0 2 0 0 0 2 2 0 2 1 0 0 1 1 2
 0 0 1 1 0 2 2 2 2 2 1 0 0 2 0 0 2 1 1 1 2 2 2]


In [23]:
print(y_test)

[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0 2 2 0 0 1 1 2
 0 0 1 1 0 2 2 2 2 2 1 0 0 2 0 0 1 1 1 1 2 1 2]


In [24]:
np.sum(y_test==y_test_pred) / np.size(y_test)

0.9333333333333333